In [39]:
import datetime

import pandas as pd
import numpy as np

import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet, Lasso

In [40]:
df_proj = pd.read_csv('transformer_data_cleaner.csv', index_col=0)
df_proj = df_proj.dropna(subset=['transformer_unit_price', 'rated_power_kVA']).copy()
df_proj.head()

,quantity,suppliers_currency,transformer_unit_price,dry_or_oil,rated_power_kVA,primary_winding,secondary_winding,no_load_losses,full_load_losses_75,full_load_losses_120,rated_volt_primary_side,filename
date,,,,,,,,,,,,
28-April-21,6.0,EUR,708000.0,Oil,19080.0,NaN,NaN,NaN,NaN,NaN,33.0,Best1 - USA - ABB - Project Hydrogen Mitshubis...
14/03/2023,4.0,EUR,235000.0,Oil,11000.0,NaN,NaN,NaN,NaN,NaN,NaN,Quote Celduc1.00.pdf
30/09/2021,5.0,EUR,267000.0,Oil,12000.0,Copper,Copper,8000.0,138000.0,146000.0,21.0,Celme_21-Celme1.pdf
30/09/2021,5.0,EUR,267000.0,Oil,12000.0,Copper,Copper,8000.0,138000.0,146000.0,21.0,Celme_21-Celme1.pdf
02.12.2020,1.0,EUR,56200.0,Dry,5000.0,Aluminium,Aluminium,5000.0,53043.0,61000.0,11.0,QT-20-Hitachi17_Rev-1.pdf


In [41]:
df = pd.read_csv('./output/v2.csv', index_col=0)
df.head()

,date,quantity,suppliers_currency,transformer_unit_price,dry_or_oil,rated_power_kVA,primary_winding,secondary_winding,no_load_loss,full_load_loss_75,full_load_loss_120,rated_volt_primary_side
0,2021-04-28,6.0,EUR,708000.0,Oil,19080.0,Copper,Copper,19002.544911,150154.155650,187698.233981,33000.0
3,2023-03-14,4.0,EUR,235000.0,Oil,11000.0,Copper,Copper,11825.516231,89149.546166,108715.888044,20000.0
8,2021-09-30,5.0,EUR,267000.0,Oil,12000.0,Copper,Copper,8000.000000,138000.000000,146000.000000,21000.0
9,2021-09-30,5.0,EUR,267000.0,Oil,12000.0,Copper,Copper,8000.000000,138000.000000,146000.000000,24000.0
10,2020-12-02,1.0,EUR,56200.0,Dry,5000.0,Aluminum,Aluminum,5000.000000,53043.000000,61000.000000,11000.0


In [42]:
df_merged = pd.merge(
    left=df_proj[['transformer_unit_price', 'rated_power_kVA', 'filename']],
    right=df,
    how='right',
    on=['transformer_unit_price', 'rated_power_kVA']
).copy()
df_merged.rename({'filename': 'project'}, axis=True, inplace=True)
df_merged = df_merged.drop_duplicates()
df_merged.head()

,transformer_unit_price,rated_power_kVA,project,date,quantity,suppliers_currency,dry_or_oil,primary_winding,secondary_winding,no_load_loss,full_load_loss_75,full_load_loss_120,rated_volt_primary_side
0,708000.0,19080.0,Best1 - USA - ABB - Project Hydrogen Mitshubis...,2021-04-28,6.0,EUR,Oil,Copper,Copper,19002.544911,150154.155650,187698.233981,33000.0
1,235000.0,11000.0,Quote Celduc1.00.pdf,2023-03-14,4.0,EUR,Oil,Copper,Copper,11825.516231,89149.546166,108715.888044,20000.0
2,267000.0,12000.0,Celme_21-Celme1.pdf,2021-09-30,5.0,EUR,Oil,Copper,Copper,8000.000000,138000.000000,146000.000000,21000.0
4,267000.0,12000.0,Celme_21-Celme1.pdf,2021-09-30,5.0,EUR,Oil,Copper,Copper,8000.000000,138000.000000,146000.000000,24000.0
6,56200.0,5000.0,QT-20-Hitachi17_Rev-1.pdf,2020-12-02,1.0,EUR,Dry,Aluminum,Aluminum,5000.000000,53043.000000,61000.000000,11000.0


In [43]:
outliers = [361500, 291000, 660000, 22000, 7300, 620000, 629000, 852000]

outlier_df = df_merged.copy()
outlier_df['outlier'] = outlier_df['transformer_unit_price'].apply(lambda x: x in outliers)

In [44]:
outlier_df.rename(
    {
        'transformer_unit_price': 'Transformer unit price in EUR',
        'rated_power_kVA': 'Rated power in kVA',
        'project': 'Project name',
        'outlier': 'Outlier'
    },
    axis=1,
    inplace=1
)

In [53]:
fig = px.scatter(
    outlier_df,
    x='Rated power in kVA',
    y='Transformer unit price in EUR',
    color='Outlier',
    hover_data='Project name'
)
# Update the layout to position the legend inside the bottom-right
fig.update_layout(
    legend=dict(
        x=1,              # Position x: Right of the plot (1 is the far right)
        y=0,              # Position y: Bottom of the plot (0 is the bottom)
        xanchor='right',   # Anchor legend's x-position to the right
        yanchor='bottom',  # Anchor legend's y-position to the bottom
        bgcolor='rgba(255, 255, 255, 0.5)'  # Optional: Set a semi-transparent background for the legend
    )
)
fig.show()

In [46]:
df = pd.get_dummies(
    df, columns=['dry_or_oil', 'primary_winding', 'secondary_winding']
).copy()

In [47]:
chosen_columns = [
    'quantity',
    'rated_power_kVA', 
    'no_load_loss', 
    # 'full_load_loss_75',
    # 'full_load_loss_120',
    'rated_volt_primary_side',
    'dry_or_oil_Dry',
    # 'dry_or_oil_Oil',
    'primary_winding_Aluminum',
    # 'primary_winding_Copper',
    'secondary_winding_Aluminum',
    # 'secondary_winding_Copper',
    # 'primary_material_price'
]

y = np.log(df['transformer_unit_price'])
# y = df['transformer_unit_price'] / df['transformer_unit_price'].mean()
X = df[chosen_columns].astype(float)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

en = ElasticNet(alpha=0.1, l1_ratio=0.5)
en.fit(X_scaled, y)

# Coefficients and Feature Importance
feature_importance = pd.DataFrame(
    {'Feature': X.columns, 
    'Lasso_Coefficient': en.coef_}
)

print(feature_importance.sort_values(by='Lasso_Coefficient'))

                      Feature  Lasso_Coefficient
4              dry_or_oil_Dry          -0.147524
6  secondary_winding_Aluminum          -0.029322
5    primary_winding_Aluminum          -0.000000
0                    quantity           0.044076
3     rated_volt_primary_side           0.161098
2                no_load_loss           0.170626
1             rated_power_kVA           0.408806


In [48]:
df['rated_volt_primary_side'].std()

np.float64(8446.190314844796)

In [49]:
df['rated_power_kVA'].std()

np.float64(5331.498402488682)

In [50]:
df['secondary_winding_Copper'].std()

np.float64(0.4170336944602445)

In [51]:
pd.DataFrame({
    'quantity': [0.0144],
    'rated_power_kVA': [0.6260],
    'rated_volt_primary_side': [0.1217],
    'alu_price_per_kg': [0.1812],
    'copper_price_per_kg': [0.0193],
    'dry_or_oil_Oil': [0.4933],
    'primary_material_price': [-0.1880],
    'secondary_material_price': [0.1803]
})

,quantity,rated_power_kVA,rated_volt_primary_side,alu_price_per_kg,copper_price_per_kg,dry_or_oil_Oil,primary_material_price,secondary_material_price
0,0.0144,0.626,0.1217,0.1812,0.0193,0.4933,-0.188,0.1803


In [52]:
px.bar(
pd.DataFrame({
    'quantity': 0.0144,
    'rated_power_kVA': 0.6260,
    'rated_volt_primary_side': 0.1217,
    'alu_price_per_kg': 0.1812,
    'copper_price_per_kg': 0.0193,
    'dry_or_oil_Oil': 0.4933,
    'primary_material_price': -0.1880,
    'secondary_material_price': 0.1803
}))

ValueError: If using all scalar values, you must pass an index